In [1]:
import logging;
logging.basicConfig(format='%(module)s||%(funcName)s||[%(process)d:%(thread)d||%(asctime)s||%(levelname)s||%(message)s', level=logging.DEBUG,
                    datefmt=logging.Formatter.default_time_format);

from rcare.readutils import DataFrameUtils as Utils
import spacy;
from rcare.nlp.customize_nlp import rcare
from spacy import displacy
s_nlp = rcare(spacy.load('en_core_web_sm')).nlp



import pandas as pd
import numpy as np
from multiprocessing import Pool

from spacy.matcher import PhraseMatcher, Matcher
from spacy.tokens import Span, Doc, Token
from sklearn.externals import joblib
import os
#CLF_MODEL = joblib.load(os.path.join("rcare/nlp/", "./section_segmenter_rcare.pickle"))
#from sklearn.cluster import KMeans
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.model_selection import train_test_split

from rcare.nlp.sections_rcare import build_annotated_model, drop_columns, DEBUG_COLUMNS, build_model, \
line_features, enrich_line_features, split_spacy_doc
import datetime
#now = datetime.datetime.now()



__init__||wrapper||[11477:139913582073600||2018-08-16 13:10:57||DEBUG||$HOME=/home/paperspace
__init__||wrapper||[11477:139913582073600||2018-08-16 13:10:57||DEBUG||matplotlib data path /home/paperspace/anaconda3/envs/en-nlp/lib/python3.6/site-packages/matplotlib/mpl-data
__init__||rc_params_from_file||[11477:139913582073600||2018-08-16 13:10:57||DEBUG||loaded rc file /home/paperspace/anaconda3/envs/en-nlp/lib/python3.6/site-packages/matplotlib/mpl-data/matplotlibrc
__init__||<module>||[11477:139913582073600||2018-08-16 13:10:57||DEBUG||matplotlib version 2.2.2
__init__||<module>||[11477:139913582073600||2018-08-16 13:10:57||DEBUG||interactive is False
__init__||<module>||[11477:139913582073600||2018-08-16 13:10:57||DEBUG||platform is linux
__init__||<module>||[11477:139913582073600||2018-08-16 13:10:57||DEBUG||loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'zipimport', 'encod

customize_nlp||__init__||[11477:139913582073600||2018-08-16 13:10:59||INFO||['tagger', 'parser', 'ner', 'CardinalRecognizer']


In [2]:
logging.info("section_build_training_data")
import os
DATA_PATH = 'data/lexpredict-contraxsuite-samples/cca_2011_Q3/text/'
SQLLITE_DIR = "/home/paperspace/dev/sqllite-node/app/data/"
DATA_DIRS =  os.listdir(DATA_PATH);

<ipython-input-2-f64530205c1a>||<module>||[7953:140716515288832||2018-08-16 11:42:21||INFO||section_build_training_data


In [ ]:

df = pd.concat([
    # Utils.read_data(path = DATA_PATH) ,
    ], ignore_index  = True)



In [4]:
import sqlite3;
import pandas as pd
import logging
from rcare.readutils import DataFrameUtils as Utils

TABLE_NAME = "features_{0}".format(datetime.datetime.now().strftime('%Y%m%d'))

logging.info("Building Features Data .. Start TABLE_NAME: %s", TABLE_NAME )       
connex  =  sqlite3.connect(SQLLITE_DIR + "/training.db")  # Opens file if exists, else creates file

#df = pd.concat([Utils.read_data(path = dir_path) ], ignore_index  = True)

test_feature_df = pd.DataFrame()
#drop_test_feature_df = pd.DataFrame()
#debug_columns = ['debug','name',  'line_id']

for path in [DATA_PATH]:
    df_g = Utils.read_data_gen(path = path, numfiles = None)    
    for index, fileData in enumerate(df_g):  
        text =  fileData.text
        name = fileData.name
        
        if(index == 0):
            if_exists = "replace"
        else: 
            if_exists = "append"


        logging.info("Index : %d %s (%s)", index, name, if_exists)
        #logging.info("%s", text.splitlines())
        
        test_feature_df = build_annotated_model(text, include_doc = {'name': name}, debug = True, using_spacy_lines = False, using_existing_model = False)
                                    

        test_feature_df.to_sql(name  = TABLE_NAME, con  = connex, if_exists = if_exists, index = False) 
        connex.commit()
        
        #break;
connex.close()
logging.info("Building Features complete")    


<ipython-input-4-eb4299f51318>||<module>||[7953:140716515288832||2018-08-16 11:43:07||INFO||Building Training Data .. Start TABLE_NAME: training_data_20180816
readutils||read_data_gen||[7953:140716515288832||2018-08-16 11:43:07||INFO||Number of files 720 in data/lexpredict-contraxsuite-samples/cca_2011_Q3/text/
readutils||read_data_gen||[7953:140716515288832||2018-08-16 11:43:07||INFO||Reading file creditcardagreement_1000.TXT
<ipython-input-4-eb4299f51318>||<module>||[7953:140716515288832||2018-08-16 11:43:07||INFO||Index : 0 creditcardagreement_1000.TXT (replace)
sections_rcare||build_model||[7953:140716515288832||2018-08-16 11:43:10||INFO||316
sections_rcare||annotated_model||[7953:140716515288832||2018-08-16 11:43:12||DEBUG||annotating the model
readutils||read_data_gen||[7953:140716515288832||2018-08-16 11:43:12||INFO||Reading file creditcardagreement_1043.TXT
<ipython-input-4-eb4299f51318>||<module>||[7953:140716515288832||2018-08-16 11:43:12||INFO||Index : 1 creditcardagreement_

sections_rcare||annotated_model||[7953:140716515288832||2018-08-16 11:44:25||DEBUG||annotating the model
readutils||read_data_gen||[7953:140716515288832||2018-08-16 11:44:25||INFO||Reading file creditcardagreement_118.TXT
<ipython-input-4-eb4299f51318>||<module>||[7953:140716515288832||2018-08-16 11:44:25||INFO||Index : 18 creditcardagreement_118.TXT (append)
sections_rcare||build_model||[7953:140716515288832||2018-08-16 11:44:31||INFO||560
sections_rcare||annotated_model||[7953:140716515288832||2018-08-16 11:44:34||DEBUG||annotating the model
readutils||read_data_gen||[7953:140716515288832||2018-08-16 11:44:34||INFO||Reading file creditcardagreement_1180.TXT
<ipython-input-4-eb4299f51318>||<module>||[7953:140716515288832||2018-08-16 11:44:34||INFO||Index : 19 creditcardagreement_1180.TXT (append)
sections_rcare||build_model||[7953:140716515288832||2018-08-16 11:44:36||INFO||107
sections_rcare||annotated_model||[7953:140716515288832||2018-08-16 11:44:37||DEBUG||annotating the model
rea

<ipython-input-4-eb4299f51318>||<module>||[7953:140716515288832||2018-08-16 11:45:58||INFO||Index : 36 creditcardagreement_1287.TXT (append)
sections_rcare||build_model||[7953:140716515288832||2018-08-16 11:45:59||INFO||77
sections_rcare||annotated_model||[7953:140716515288832||2018-08-16 11:45:59||DEBUG||annotating the model
readutils||read_data_gen||[7953:140716515288832||2018-08-16 11:45:59||INFO||Reading file creditcardagreement_1288.TXT
<ipython-input-4-eb4299f51318>||<module>||[7953:140716515288832||2018-08-16 11:45:59||INFO||Index : 37 creditcardagreement_1288.TXT (append)
sections_rcare||build_model||[7953:140716515288832||2018-08-16 11:46:00||INFO||76
sections_rcare||annotated_model||[7953:140716515288832||2018-08-16 11:46:00||DEBUG||annotating the model
readutils||read_data_gen||[7953:140716515288832||2018-08-16 11:46:00||INFO||Reading file creditcardagreement_1291.TXT
<ipython-input-4-eb4299f51318>||<module>||[7953:140716515288832||2018-08-16 11:46:00||INFO||Index : 38 credi

sections_rcare||annotated_model||[7953:140716515288832||2018-08-16 11:47:29||DEBUG||annotating the model
readutils||read_data_gen||[7953:140716515288832||2018-08-16 11:47:30||INFO||Reading file creditcardagreement_1444.TXT
<ipython-input-4-eb4299f51318>||<module>||[7953:140716515288832||2018-08-16 11:47:30||INFO||Index : 55 creditcardagreement_1444.TXT (append)
sections_rcare||build_model||[7953:140716515288832||2018-08-16 11:47:32||INFO||232
sections_rcare||annotated_model||[7953:140716515288832||2018-08-16 11:47:33||DEBUG||annotating the model
readutils||read_data_gen||[7953:140716515288832||2018-08-16 11:47:33||INFO||Reading file creditcardagreement_1448.TXT
<ipython-input-4-eb4299f51318>||<module>||[7953:140716515288832||2018-08-16 11:47:33||INFO||Index : 56 creditcardagreement_1448.TXT (append)
sections_rcare||build_model||[7953:140716515288832||2018-08-16 11:47:34||INFO||54
sections_rcare||annotated_model||[7953:140716515288832||2018-08-16 11:47:35||DEBUG||annotating the model
re

<ipython-input-4-eb4299f51318>||<module>||[7953:140716515288832||2018-08-16 11:49:53||INFO||Index : 73 creditcardagreement_1612.TXT (append)
sections_rcare||build_model||[7953:140716515288832||2018-08-16 11:49:56||INFO||262
sections_rcare||annotated_model||[7953:140716515288832||2018-08-16 11:49:57||DEBUG||annotating the model
readutils||read_data_gen||[7953:140716515288832||2018-08-16 11:49:57||INFO||Reading file creditcardagreement_1613.TXT
<ipython-input-4-eb4299f51318>||<module>||[7953:140716515288832||2018-08-16 11:49:57||INFO||Index : 74 creditcardagreement_1613.TXT (append)
sections_rcare||build_model||[7953:140716515288832||2018-08-16 11:50:01||INFO||331
sections_rcare||annotated_model||[7953:140716515288832||2018-08-16 11:50:02||DEBUG||annotating the model
readutils||read_data_gen||[7953:140716515288832||2018-08-16 11:50:02||INFO||Reading file creditcardagreement_1627.TXT
<ipython-input-4-eb4299f51318>||<module>||[7953:140716515288832||2018-08-16 11:50:02||INFO||Index : 75 cre

...

readutils||read_data_gen||[7953:140716515288832||2018-08-16 12:42:22||INFO||Reading file creditcardagreement_3737.TXT
<ipython-input-4-eb4299f51318>||<module>||[7953:140716515288832||2018-08-16 12:42:22||INFO||Index : 628 creditcardagreement_3737.TXT (append)
sections_rcare||build_model||[7953:140716515288832||2018-08-16 12:42:23||INFO||92
sections_rcare||annotated_model||[7953:140716515288832||2018-08-16 12:42:24||DEBUG||annotating the model
readutils||read_data_gen||[7953:140716515288832||2018-08-16 12:42:24||INFO||Reading file creditcardagreement_3747.TXT
<ipython-input-4-eb4299f51318>||<module>||[7953:140716515288832||2018-08-16 12:42:24||INFO||Index : 629 creditcardagreement_3747.TXT (append)
sections_rcare||build_model||[7953:140716515288832||2018-08-16 12:42:29||INFO||394
sections_rcare||annotated_model||[7953:140716515288832||2018-08-16 12:42:30||DEBUG||annotating the model
readutils||read_data_gen||[7953:140716515288832||2018-08-16 12:42:31||INFO||Reading file creditcardagreem

sections_rcare||build_model||[7953:140716515288832||2018-08-16 12:43:55||INFO||173
sections_rcare||annotated_model||[7953:140716515288832||2018-08-16 12:43:55||DEBUG||annotating the model
readutils||read_data_gen||[7953:140716515288832||2018-08-16 12:43:56||INFO||Reading file creditcardagreement_3970.TXT
<ipython-input-4-eb4299f51318>||<module>||[7953:140716515288832||2018-08-16 12:43:56||INFO||Index : 647 creditcardagreement_3970.TXT (append)
sections_rcare||build_model||[7953:140716515288832||2018-08-16 12:43:58||INFO||161
sections_rcare||annotated_model||[7953:140716515288832||2018-08-16 12:43:59||DEBUG||annotating the model
readutils||read_data_gen||[7953:140716515288832||2018-08-16 12:43:59||INFO||Reading file creditcardagreement_3971.TXT
<ipython-input-4-eb4299f51318>||<module>||[7953:140716515288832||2018-08-16 12:43:59||INFO||Index : 648 creditcardagreement_3971.TXT (append)
sections_rcare||build_model||[7953:140716515288832||2018-08-16 12:44:01||INFO||167
sections_rcare||annot

readutils||read_data_gen||[7953:140716515288832||2018-08-16 12:45:09||INFO||Reading file creditcardagreement_4260.TXT
<ipython-input-4-eb4299f51318>||<module>||[7953:140716515288832||2018-08-16 12:45:09||INFO||Index : 665 creditcardagreement_4260.TXT (append)
sections_rcare||build_model||[7953:140716515288832||2018-08-16 12:45:11||INFO||161
sections_rcare||annotated_model||[7953:140716515288832||2018-08-16 12:45:13||DEBUG||annotating the model
readutils||read_data_gen||[7953:140716515288832||2018-08-16 12:45:13||INFO||Reading file creditcardagreement_4292.TXT
<ipython-input-4-eb4299f51318>||<module>||[7953:140716515288832||2018-08-16 12:45:13||INFO||Index : 666 creditcardagreement_4292.TXT (append)
sections_rcare||build_model||[7953:140716515288832||2018-08-16 12:45:18||INFO||531
sections_rcare||annotated_model||[7953:140716515288832||2018-08-16 12:45:21||DEBUG||annotating the model
readutils||read_data_gen||[7953:140716515288832||2018-08-16 12:45:21||INFO||Reading file creditcardagree

sections_rcare||build_model||[7953:140716515288832||2018-08-16 12:46:15||INFO||114
sections_rcare||annotated_model||[7953:140716515288832||2018-08-16 12:46:16||DEBUG||annotating the model
readutils||read_data_gen||[7953:140716515288832||2018-08-16 12:46:16||INFO||Reading file creditcardagreement_851.TXT
<ipython-input-4-eb4299f51318>||<module>||[7953:140716515288832||2018-08-16 12:46:16||INFO||Index : 684 creditcardagreement_851.TXT (append)
sections_rcare||build_model||[7953:140716515288832||2018-08-16 12:46:18||INFO||119
sections_rcare||annotated_model||[7953:140716515288832||2018-08-16 12:46:19||DEBUG||annotating the model
readutils||read_data_gen||[7953:140716515288832||2018-08-16 12:46:19||INFO||Reading file creditcardagreement_852.TXT
<ipython-input-4-eb4299f51318>||<module>||[7953:140716515288832||2018-08-16 12:46:19||INFO||Index : 685 creditcardagreement_852.TXT (append)
sections_rcare||build_model||[7953:140716515288832||2018-08-16 12:46:25||INFO||531
sections_rcare||annotated

readutils||read_data_gen||[7953:140716515288832||2018-08-16 12:47:51||INFO||Reading file creditcardagreement_877.TXT
<ipython-input-4-eb4299f51318>||<module>||[7953:140716515288832||2018-08-16 12:47:51||INFO||Index : 702 creditcardagreement_877.TXT (append)
sections_rcare||build_model||[7953:140716515288832||2018-08-16 12:47:54||INFO||277
sections_rcare||annotated_model||[7953:140716515288832||2018-08-16 12:47:56||DEBUG||annotating the model
readutils||read_data_gen||[7953:140716515288832||2018-08-16 12:47:56||INFO||Reading file creditcardagreement_880.TXT
<ipython-input-4-eb4299f51318>||<module>||[7953:140716515288832||2018-08-16 12:47:56||INFO||Index : 703 creditcardagreement_880.TXT (append)
sections_rcare||build_model||[7953:140716515288832||2018-08-16 12:47:57||INFO||110
sections_rcare||annotated_model||[7953:140716515288832||2018-08-16 12:47:58||DEBUG||annotating the model
readutils||read_data_gen||[7953:140716515288832||2018-08-16 12:47:58||INFO||Reading file creditcardagreement

In [ ]:
#test_feature_df['target1'] = 1
test_feature_df[['debug','line_is_start_of_sent_0', 'line_is_end_of_sent_0', 'line_id']]

In [2]:
text = Utils.read_file_data('data/lexpredict-contraxsuite-samples/cca_2011_Q3/text/creditcardagreement_2624.TXT')
lines = text.splitlines()

In [ ]:
s_lines = list(map(lambda line : s_nlp(line.strip()) if(len(line.strip()) > 0 ) else s_nlp(" ") , lines))